In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
# from ESO import ESO

c:\Users\SalnikovMA\Anaconda3\envs\Torch_Env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Reverse_Regression(nn.Module):
    def __init__(self, model):
        super().__init__()
        input_size, output_size = model.weights.shape[0], model.weights.shape[1]
        self.opt_vec = nn.Parameter(torch.randn(output_size, input_size))

    def forward(self, weights, weights_2, weights_3, bias):
        x3 = self.opt_vec**3 @ weights_3
        x2 = self.opt_vec**2 @ weights_2
        x1 = self.opt_vec @ weights
        return x3 + x2 + x1 + bias


class Regression(nn.Module):
        def __init__(self, input_size, output_size, lambda_):
            super().__init__()
            self.weights = nn.Parameter(torch.randn(input_size, output_size))
            self.weights_2 = nn.Parameter(torch.randn(input_size, output_size))
            self.weights_3 = nn.Parameter(torch.randn(input_size, output_size))
            # self.weights_3 = nn.Parameter(torch.randn(input_size, input_size))
            self.bias = nn.Parameter(torch.randn(output_size))
            self.lambda_ = lambda_
            # self.activation = nn.Tanh()  # добавляем функцию активации
            # self.activation = nn.Mish() 

        def forward(self, x):
            # x = x @ self.weights_2 # применяем первый слой
            # x = self.activation(x)  # применяем функцию активации
            return x**3 @ self.weights_3 + x**2 @ self.weights_2 + x @ self.weights + self.bias

        def l1_reg(self):
            return self.lambda_ * (torch.sum(torch.abs(self.weights)) + torch.sum(torch.abs(self.weights_2)) + torch.sum(torch.abs(self.weights_3)) + torch.sum(torch.abs(self.bias)))

        def l2_reg(self):
            return self.lambda_ * torch.sum(torch.pow(self.weights, 2))


In [3]:
df = pd.read_excel('data/Global_Unc6_OF_v23_structured.xlsx')

X = df.iloc[3:, 2:16]
Y = df.iloc[:, 32:]
new_header_1 = Y.iloc[1]
new_header_2 = Y.iloc[2]
new_header = [str(i) + ' / ' + str(j) for i, j in zip(new_header_1, new_header_2)]
Y = Y[3:] #take the data less the header row
Y.columns = new_header #set the header row as the df header


scaler = MinMaxScaler()
inputs = scaler.fit_transform(X)#.to_numpy(dtype=float)

In [5]:
feature_names = df.iloc[2:3, 2:16].loc[2].to_list()
lim_df = pd.read_excel('data/input_limits.xlsx')
min_val = scaler.transform(np.array([[lim_df[lim_df['Parameter']==f_n].Min.values[0] for f_n in feature_names]]))
max_val = scaler.transform(np.array([[lim_df[lim_df['Parameter']==f_n].Max.values[0] for f_n in feature_names]]))
min_val = torch.from_numpy(min_val).float()
max_val = torch.from_numpy(max_val).float()

C:\Users\SalnikovMA\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\SalnikovMA\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [6]:
n = 0
for feature in tqdm(Y.columns):
    targets = Y[feature].to_numpy(dtype=float)

    X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.3, random_state=42)

    X_train, X_test = torch.from_numpy(X_train).float(), torch.from_numpy(X_test).float()  
    y_train, y_test = torch.from_numpy(y_train).float().view(-1, 1), torch.from_numpy(y_test).float().view(-1, 1)

    # Инициализируем модель
    input_size = X_train.shape[1]
    output_size = 1
    lambda_ = 0.01
    model = Regression(input_size, output_size, lambda_)

    # Инициализируем фуункцию потерь и оптимизатор
    # criterion = nn.MSELoss()

    criterion = nn.L1Loss()

    optimizer = optim.LBFGS(model.parameters(), lr=0.01)

    # эпоха обучения
    def fitness_step():
        global loss
        outputs = model(X_train) # Получаем предсказания
        loss =  criterion(outputs, y_train) # Обсчитываем функцию потерь
        if reg == 'l1':
            loss += model.l1_reg() # Добавляем L1 регуляризацию
        elif reg == 'l2':
            loss += model.l2_reg() # Добавляем L2 регуляризацию

        # Выполняем оптимизацию параметров модели
        optimizer.zero_grad()
        loss.backward()
        #print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}') 
        return loss


    # Запускаем обучение
    reg = 'l1'
    num_epochs = 500
    for epoch in range(num_epochs):
        optimizer.step(fitness_step)
    #print(f' Total Loss: {loss.item():.4f}')

    print(f'L1Loss модели на обучающей выборке {criterion(model(X_train), y_train)}')
    print(f'L1Loss модели на тестовой выборке {criterion(model(X_test), y_test)}')


    # подбираем ручки по полученной модели
    reverse_model = Reverse_Regression(model)
    optimizer = optim.Adam(reverse_model.parameters(), lr=0.01)

    weights_3 = torch.from_numpy(model.weights_3.detach().numpy()).float()
    weights_2 = torch.from_numpy(model.weights_2.detach().numpy()).float()
    weights = torch.from_numpy(model.weights.detach().numpy()).float()
    bias = torch.from_numpy(model.bias.detach().numpy()).float()

    y_ = torch.from_numpy(np.array([[0]])).float().view(-1, 1)

    num_epochs = 1000
    for epoch in range(num_epochs):
        # Forward pass
        outputs = reverse_model(weights, weights_2, weights_3, bias) # Получаем предсказания
        loss = criterion(outputs, y_) # Обсчитываем функцию потерь

        # Выполняем оптимизацию параметров модели
        optimizer.zero_grad()
        loss.backward()

        # Ограничиваем значения оптимизируемых параметров
        reverse_model.opt_vec.data = torch.clamp(reverse_model.opt_vec.data, min_val, max_val)

        # Выполняем шаг оптимизации
        optimizer.step()

    # Ограничиваем значения оптимизируемых параметров

    reverse_model.opt_vec.data = torch.clamp(reverse_model.opt_vec.data, min_val, max_val)

    outputs = reverse_model(weights, weights_2, weights_3, bias) # Получаем предсказания
    loss = criterion(outputs, y_)

    print(f'Loss with opt features: {loss.item():.4f}')
    
    weights = np.concatenate((model.weights_3.view(-1,).detach().numpy(),
                              model.weights_2.view(-1,).detach().numpy(),
                              model.weights.view(-1,).detach().numpy(),
                              model.bias.view(-1,).detach().numpy(),
                              scaler.inverse_transform(reverse_model.opt_vec.detach().numpy())[0],
                              np.array([loss.item()])), axis=None)

    if n == 0:
        df_res = pd.DataFrame()
        df_res[feature] = weights
        df_res.to_csv('DNN_weights.csv', index=False)
    
    else:
        df_res = pd.read_csv('DNN_weights.csv')
        df_res[feature] = weights
        df_res.to_csv('DNN_weights.csv', index=False)
        
    n = 1

  0%|          | 0/927 [00:00<?, ?it/s]

L1Loss модели на обучающей выборке 1.543007731437683
L1Loss модели на тестовой выборке 2.121776819229126


  0%|          | 1/927 [00:07<1:53:38,  7.36s/it]

Loss with opt features: 0.0020


  0%|          | 2/927 [00:13<1:45:36,  6.85s/it]

...

Loss with opt features: 0.0110
L1Loss модели на обучающей выборке 1.6438992023468018
L1Loss модели на тестовой выборке 2.0258305072784424
Loss with opt features: 0.1103


100%|█████████▉| 926/927 [2:03:43<00:08,  8.56s/it]

L1Loss модели на обучающей выборке 0.63884037733078
L1Loss модели на тестовой выборке 1.0080273151397705


100%|██████████| 927/927 [2:03:51<00:00,  8.02s/it]

Loss with opt features: 1.0452


In [ ]:

feature = Y.columns[0]

# получаем датасет из библиотеки sklearn
scaler = MinMaxScaler()

inputs = scaler.fit_transform(X)#.to_numpy(dtype=float)
targets = Y[feature].to_numpy(dtype=float)

X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.3, random_state=42)

X_train, X_test = torch.from_numpy(X_train).float(), torch.from_numpy(X_test).float()  
y_train, y_test = torch.from_numpy(y_train).float().view(-1, 1), torch.from_numpy(y_test).float().view(-1, 1)

# Инициализируем модель
input_size = X_train.shape[1]
output_size = 1
lambda_ = 0.01
model = Regression(input_size, output_size, lambda_)

# Инициализируем фуункцию потерь и оптимизатор
# criterion = nn.MSELoss()

criterion = nn.L1Loss()

optimizer = optim.LBFGS(model.parameters(), lr=0.01)

# эпоха обучения
def fitness_step():
    global loss
    outputs = model(X_train) # Получаем предсказания
    loss =  criterion(outputs, y_train) # Обсчитываем функцию потерь
    if reg == 'l1':
        loss += model.l1_reg() # Добавляем L1 регуляризацию
    elif reg == 'l2':
        loss += model.l2_reg() # Добавляем L2 регуляризацию

    # Выполняем оптимизацию параметров модели
    optimizer.zero_grad()
    loss.backward()
    #print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}') 
    return loss


# Запускаем обучение
reg = 'l1'
num_epochs = 500
for epoch in range(num_epochs):
    optimizer.step(fitness_step)
#print(f' Total Loss: {loss.item():.4f}')

print(f'L1Loss модели на обучающей выборке {criterion(model(X_train), y_train)}')
print(f'L1Loss модели на тестовой выборке {criterion(model(X_test), y_test)}')


In [ ]:
feature_names = df.iloc[2:3, 2:16].loc[2].to_list()
lim_df = pd.read_excel('data/input_limits.xlsx')
min_val = scaler.transform(np.array([[lim_df[lim_df['Parameter']==f_n].Min.values[0] for f_n in feature_names]]))
max_val = scaler.transform(np.array([[lim_df[lim_df['Parameter']==f_n].Max.values[0] for f_n in feature_names]]))
min_val = torch.from_numpy(min_val).float()
max_val = torch.from_numpy(max_val).float()

In [ ]:
reverse_model = Reverse_Regression(model)
criterion = nn.L1Loss()

optimizer = optim.Adam(reverse_model.parameters(), lr=0.01)

weights_3 = torch.from_numpy(model.weights_3.detach().numpy()).float()
weights_2 = torch.from_numpy(model.weights_2.detach().numpy()).float()
weights = torch.from_numpy(model.weights.detach().numpy()).float()
bias = torch.from_numpy(model.bias.detach().numpy()).float()

y_ = torch.from_numpy(np.array([[0]])).float().view(-1, 1)



num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = reverse_model(weights, weights_2, weights_3, bias) # Получаем предсказания
    loss = criterion(outputs, y_) # Обсчитываем функцию потерь

    # Выполняем оптимизацию параметров модели
    optimizer.zero_grad()
    loss.backward()

    # Ограничиваем значения оптимизируемых параметров
    reverse_model.opt_vec.data = torch.clamp(reverse_model.opt_vec.data, min_val, max_val)

    # Выполняем шаг оптимизации
    optimizer.step()

    # Print the loss
    if (epoch+1) % 5== 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Ограничиваем значения оптимизируемых параметров

reverse_model.opt_vec.data = torch.clamp(reverse_model.opt_vec.data, min_val, max_val)

outputs = reverse_model(weights, weights_2, weights_3, bias) # Получаем предсказания
loss = criterion(outputs, y_)

print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
reverse_model.opt_vec, scaler.inverse_transform(reverse_model.opt_vec.detach().numpy())

In [ ]:
 scaler.inverse_transform(reverse_model.opt_vec.detach().numpy())[0]